In [1]:
# test - tree construction
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print

urdf_file = "./assets/kuka_iiwa.urdf"
root, links, robot = rmp_tree_from_urdf(urdf_file)

rmp_tree_print(root)

q = np.array([1,2,3,4,5,6,7])
dq = np.array([-1,-2,-3,-4,-5,-6,-7])

root.set_root_state(q,dq)
root.pushforward()


`- root
   |- proj_q_lbr_iiwa_link_1
   |  `- lbr_iiwa_link_1
   |- proj_q_lbr_iiwa_link_2
   |  `- lbr_iiwa_link_2
   |- proj_q_lbr_iiwa_link_3
   |  `- lbr_iiwa_link_3
   |- proj_q_lbr_iiwa_link_4
   |  `- lbr_iiwa_link_4
   |- proj_q_lbr_iiwa_link_5
   |  `- lbr_iiwa_link_5
   |- proj_q_lbr_iiwa_link_6
   |  `- lbr_iiwa_link_6
   `- proj_q_lbr_iiwa_link_7
      `- lbr_iiwa_link_7
root: pushforward
proj_q_lbr_iiwa_link_1: pushforward
proj_q_lbr_iiwa_link_1: x = [1.], dx = [-1.]
lbr_iiwa_link_1: pushforward
lbr_iiwa_link_1: x = [0.     0.     0.1575 1.     0.     0.    ], dx = [ 0.  0.  0.  0.  0. -1.]
proj_q_lbr_iiwa_link_2: pushforward
proj_q_lbr_iiwa_link_2: x = [1. 2.], dx = [-1. -2.]
lbr_iiwa_link_2: pushforward
lbr_iiwa_link_2: x = [ 0.          0.          0.36       -1.14159265 -1.         -1.57079633], dx = [ 0.          0.          0.          1.68294197 -1.08060461 -1.        ]
proj_q_lbr_iiwa_link_3: pushforward
proj_q_lbr_iiwa_link_3: x = [1. 2. 3.], dx = [-1. -2. -3.]
lb

In [4]:
import numpy as np
from rmpflow.rmp.rbd_rmp import *
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

from scipy.spatial.transform import Rotation as R

ax, ay, az = math.pi/2, math.pi/3, math.pi/4

# euler zyx
rot_sv = s.RotZ(ax)*s.RotY(ay)*s.RotX(az)
rot = rot_sv.transpose()
r = R.from_matrix(np.array(rot))
print("Expected ax, ay, az = {}, {}, {}".format(ax,ay,az))
print(r.as_euler('zyx', degrees=False))


Expected ax, ay, az = 1.5707963267948966, 1.0471975511965976, 0.7853981633974483
[1.57079633 1.04719755 0.78539816]


In [1]:
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, PositionProjection

root, links = rmp_tree_from_urdf("./assets/kuka_iiwa.urdf")

# link5_pos = PositionProjection("link5_pos", links[''])
for key, value in links.items():
    print(key, " : ", value.name, ", ", value.parent.name, ", ", value.parent.param_map)


lbr_iiwa_link_1  :  lbr_iiwa_link_1 ,  proj_lbr_iiwa_link_1 ,  [1 0 0 0 0 0 0]
lbr_iiwa_link_2  :  lbr_iiwa_link_2 ,  proj_lbr_iiwa_link_2 ,  [1 1 0 0 0 0 0]
lbr_iiwa_link_3  :  lbr_iiwa_link_3 ,  proj_lbr_iiwa_link_3 ,  [1 1 1 0 0 0 0]
lbr_iiwa_link_4  :  lbr_iiwa_link_4 ,  proj_lbr_iiwa_link_4 ,  [1 1 1 1 0 0 0]
lbr_iiwa_link_5  :  lbr_iiwa_link_5 ,  proj_lbr_iiwa_link_5 ,  [1 1 1 1 1 0 0]
lbr_iiwa_link_6  :  lbr_iiwa_link_6 ,  proj_lbr_iiwa_link_6 ,  [1 1 1 1 1 1 0]
lbr_iiwa_link_7  :  lbr_iiwa_link_7 ,  proj_lbr_iiwa_link_7 ,  [1 1 1 1 1 1 1]


In [1]:
import pybullet as b
import time
import pybullet_data

import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print, node_array, ProjectionNode, PositionProjection, FrameProjection
import rmpflow.rmp.rmp_leaf as leaves

# constants
DT = 0.001 #s
urdf_file = "./assets/kuka_iiwa.urdf"

# import rmp tree
root, links, robot = rmp_tree_from_urdf(urdf_file)

# attach frame projector and attractor
proj_node = PositionProjection("proj_f_lbr_iiwa_link_7", links["lbr_iiwa_link_7"])
target_pos = np.array([0.5,0.5,0.7])
atrc = leaves.GoalAttractorUni("attractor", proj_node, target_pos.transpose(), gain=0, eta=0)

# tree printing
# rmp_tree_print(root)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath())
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
plane_id = b.loadURDF("plane.urdf")
start_pos = [0,0,0]
start_ori = b.getQuaternionFromEuler([0,0,0])
robot_id = b.loadURDF(urdf_file,start_pos, start_ori, flags=(b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS))
n_dof = b.getNumJoints(robot_id)
joints_id = range(n_dof) 

# setup configuration
b.setJointMotorControlArray(robot_id, joints_id, b.VELOCITY_CONTROL, forces=[0] * n_dof)

q = np.array([1,2,3,4,5,6,7])
dq = np.array([-1,-2,-3,-4,-5,-6,-7])
# dq = np.array([0,0,0,0,0,0,0])
root.solve(q,dq)

# # simulate
# for i in range(0, 10000):
#     joint_states = b.getJointStates(robot_id, joints_id)
    
#     # read state feedback
#     q = np.array([joint_states[i][0] for i in joints_id])
#     dq = np.array([joint_states[i][1] for i in joints_id])
#     ddq = root.solve(q,dq)

#     print(ddq)

#     # compute torque and dynamics
#     M, _, h = links["lbr_iiwa_link_7"].update_dynamics()
#     tau = (M.dot(ddq) + h).flatten()

#     # command system:
#     for i in joints_id:
#         b.setJointMotorControl2(robot_id, i, b.TORQUE_CONTROL, force=tau[i])

#     # sleep
#     b.stepSimulation()
#     time.sleep(DT)


root: pushforward
proj_q_lbr_iiwa_link_1: pushforward
proj_q_lbr_iiwa_link_1: x = [1.], dx = [-1.]
lbr_iiwa_link_1: pushforward
lbr_iiwa_link_1: x = [0.     0.     0.1575 1.     0.     0.    ], dx = [ 0.  0.  0.  0.  0. -1.]
proj_q_lbr_iiwa_link_2: pushforward
proj_q_lbr_iiwa_link_2: x = [1. 2.], dx = [-1. -2.]
lbr_iiwa_link_2: pushforward
lbr_iiwa_link_2: x = [ 0.          0.          0.36       -1.14159265 -1.         -1.57079633], dx = [ 0.          0.          0.          1.68294197 -1.08060461 -1.        ]
proj_q_lbr_iiwa_link_3: pushforward
proj_q_lbr_iiwa_link_3: x = [1. 2. 3.], dx = [-1. -2. -3.]
lbr_iiwa_link_3: pushforward
lbr_iiwa_link_3: x = [ 0.10046993  0.15647264  0.27489797 -1.45128424  0.51357651 -2.0689272 ], dx = [ 0.24843429  0.04275185  0.37190265  0.20905548 -3.37604682  0.24844051]
proj_q_lbr_iiwa_link_4: pushforward
proj_q_lbr_iiwa_link_4: x = [1. 2. 3. 4.], dx = [-1. -2. -3. -4.]
lbr_iiwa_link_4: pushforward
lbr_iiwa_link_4: x = [ 0.20634411  0.32136191  0.1852

array([ 7.88338769e+00, -4.83718327e+01, -1.79848111e+00,  1.80940725e+01,
       -8.54688037e-16,  0.00000000e+00,  0.00000000e+00])